<a href="https://colab.research.google.com/github/Greeshu-09/AISECT-AIML/blob/main/CV_Randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the Libraies
import numpy as np
import pandas as pd

In [ ]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [ ]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [ ]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [ ]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [ ]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_predict

#Small parameter grid for speed
param_grid = {
    'n_estimators': [50, 100],          # fewer trees → faster
    'max_depth': [None, 5],             # shallow trees train quicker
    'min_samples_split': [2, 5],        # small values only
    'min_samples_leaf': [1, 2],         # minimal values
    'criterion': ['gini'],              # skip entropy for speed
    'class_weight': [None]              # skip balanced to reduce runs
}

# GridSearchCV setup
grid = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=3,          # use 3-fold CV instead of 5 for speed
    refit=True,
    verbose=2,
    n_jobs=-1
)

# Fit the model
grid.fit(X, y)
# Best parameters & score
print("Best Parameters:", grid.best_params_)
print("Best f1_weighted Score:", grid.best_score_)

# Best estimator for predictions
best_knn = grid.best_estimator_

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best Parameters: {'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Best f1_weighted Score: 0.8900755254494493


In [ ]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=3)

In [ ]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [ ]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
The f1_macro: 0.8904805306870887
The confusion Matrix:
 [[232  25]
 [ 19 124]]
The report:
               precision    recall  f1-score   support

           0       0.92      0.90      0.91       257
           1       0.83      0.87      0.85       143

    accuracy                           0.89       400
   macro avg       0.88      0.88      0.88       400
weighted avg       0.89      0.89      0.89       400



In [ ]:
# Save Best Model
import pickle
filename = "Randomforestclassifier.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [ ]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.262374,0.074052,0.056760,0.016642,None,gini,None,1,2,50,"{'class_weight': None, 'criterion': 'gini', 'm...",0.867657,0.891363,0.793223,0.850747,0.041812,16
1,0.463497,0.083465,0.070399,0.013302,None,gini,None,1,2,100,"{'class_weight': None, 'criterion': 'gini', 'm...",0.868738,0.900235,0.793223,0.854065,0.044903,15
2,0.308083,0.046455,0.049983,0.015458,None,gini,None,1,5,50,"{'class_weight': None, 'criterion': 'gini', 'm...",0.885053,0.916138,0.808205,0.869799,0.045364,13
3,0.584456,0.063266,0.077647,0.013925,None,gini,None,1,5,100,"{'class_weight': None, 'criterion': 'gini', 'm...",0.860457,0.923996,0.793223,0.859225,0.053395,14
4,0.355114,0.050506,0.045705,0.021325,None,gini,None,2,2,50,"{'class_weight': None, 'criterion': 'gini', 'm...",0.924143,0.931386,0.778108,0.877879,0.070611,11


In [ ]:
# Get user input
age = int(input("Enter Age: "))
salary = float(input("Enter Estimated Salary: "))
gender_male = int(input("Enter Gender (0 for Female, 1 for Male): "))

# Preprocess input
# The order of features should match the training data: 'Age', 'EstimatedSalary', 'Gender_Male'
user_input = np.array([[age, salary, gender_male]])
user_input_scaled = sc.transform(user_input)

# Load the saved model
loaded_model = pickle.load(open(filename, 'rb'))

# Make prediction
prediction = loaded_model.predict(user_input_scaled)

# Display the prediction
if prediction[0] == 1:
    print("The user is likely to purchase.")
else:
    print("The user is unlikely to purchase.")

Enter Age: 36
Enter Estimated Salary: 33000
Enter Gender (0 for Female, 1 for Male): 1
The user is unlikely to purchase.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
